#Загрузка данных + импорт библиотек

In [ ]:
from google.colab import drive


drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd


df = pd.read_csv('drive/My Drive/morze_data/trainDataSet_morze_about_all_intents.csv', sep=',')

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
df.head()

,text,label
0,ты мне очень помогла,good
1,ты лучшая,good
2,настоящий профессионал,good
3,прекрасно сработано,good
4,ты хороша в этом,good


In [ ]:
train_list = df.text.to_list()
label_list = df.label.to_list()

In [ ]:
!pip install torch numpy pandas onnxruntime-gpu onnx transformers optimum evaluate datasets sentence-transformers --quiet

In [ ]:
import sklearn
import numpy as np
from scipy.spatial.distance import cdist
# from sentence_transformers import SentenceTransformer
from optimum.onnxruntime import ORTModelForSequenceClassification, ORTModelForFeatureExtraction
# from drive.MyDrive.morze_data.morze_about_answers import answers
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel

#MyClass

In [ ]:
class CustomSentenceTransformer():
    def __init__(self, model, tokenizer):
        super().__init__()
        self.model = model
        self.tokenizer = tokenizer

    def encode(self, text):
        t = self.tokenizer(text, padding=True, truncation=True, return_tensors='pt')
        with torch.no_grad():
            model_output = self.model(**{k: v.to(self.model.device) for k, v in t.items()})
        embeddings = model_output.last_hidden_state[:, 0, :]
        embeddings = torch.nn.functional.normalize(embeddings)
        self.embeddings = embeddings.cpu().numpy()
        return self.embeddings

In [ ]:
#@title TEST Custom ST
# from sentence_transformers import SentenceTransformer

tokenizer = AutoTokenizer.from_pretrained("yuraz28/bert-multilingual-passage-reranking-msmarco-onnx-fe-optimized-fp16")
model = ORTModelForFeatureExtraction.from_pretrained("yuraz28/bert-multilingual-passage-reranking-msmarco-onnx-fe-optimized-fp16", use_io_binding=True ).to(device)

# st = CustomSentenceTransformer(model, tokenizer)
# print(st.encode(["привет, как дела?"]).shape)

# print(SentenceTransformer("bert-base-uncased").encode(["привет, как дела"]).shape)

The ONNX file optimized_fp16.onnx is not a regular name used in optimum.onnxruntime, the ORTModel might not behave as expected.


In [ ]:
types = ['sentence-transformers', 'nli']

class Model_searh():
    def __init__(self, path_models: str, typee: str = 'sentence-transformers') -> None:
        self.typee = typee
        if typee == 'sentence-transformers':
            self.model = SentenceTransformer(path_models)
        elif typee == 'nli':
            self.tokenizer = AutoTokenizer.from_pretrained(path_models)
            self.model = AutoModelForSequenceClassification.from_pretrained(path_models)
            if torch.cuda.is_available():
                self.model.cuda()

    def fit(self, train_list: list) -> None:
        if self.typee == 'sentence-transformers':
            self.train_list = train_list
            self.embeddings = self.model.encode(train_list)

    def get_best(self, query: str, label_list: np.array, *, max_value: int = 10, K: int = 3, label: str = 'entailment', normalize: bool = True) -> dict:

        if self.typee == 'sentence-transformers':
            self.query_embedding = self.model.encode([query])
            self.distances = np.array(cdist(self.query_embedding, self.embeddings, "cosine")[0])
            self.ind = np.argsort(self.distances, axis=0)
            self.distances = self.distances[self.ind]
            self.label_list = np.array(label_list)[self.ind]
            self.mask = np.isin(self.distances, self.distances[self.distances < max(max_value, min(self.distances) + 0.01)])

            return {
                'input_text': query,
                'output_texts': self.label_list[self.mask][:K],
                'output_distances': self.distances[self.mask][:K]
            }
            # return {
            #     'input_text': query,
            #     'output_texts': [label_list[self.ind[i]] for i in range(K)],
            #     'output_distances': [self.distances[self.ind[i]] for i in range(K)]
            # }

        elif self.typee == 'nli':
            self.tokens = self.tokenizer([query] * len(label_list), label_list, truncation=True, return_tensors='pt',
                               padding=True)
            with torch.inference_mode():
                self.result = torch.softmax(self.model(**self.tokens.to(self.model.device)).logits, -1)
            self.proba = self.result[:, self.model.config.label2id[label]].cpu().numpy()

            if normalize:
                self.proba /= sum(self.proba)

            self.ind = np.argsort(self.proba, axis=0)[::-1]
            self.label_list = np.array(label_list)[self.ind]
            self.proba = np.array(self.proba)[self.ind]
            self.mask = np.isin(self.proba, self.proba[self.proba < max(max_value, min(self.proba) + 0.01)])

            return {
                  'input_text': query,
                  'output_texts': self.label_list,
                  'output_distances': self.proba
            }


#Models

##Oplimize model

In [ ]:
st = CustomSentenceTransformer(model, tokenizer)

In [ ]:
embeddings = st.encode(train_list)

In [ ]:
def get_best(query: str, label_list: np.array, model, max_value: int = 10, K: int = 3):
        query_embedding = model.encode([query])
        distances = np.array(cdist(query_embedding, embeddings, "cosine")[0])
        ind = np.argsort(distances, axis=0)
        distances = distances[ind]
        label_list = np.array(label_list)[ind]
        mask = np.isin(distances, distances[distances < max(max_value, min(distances) + 0.01)])

        return {
            'input_text': query,
            'output_texts': label_list[mask][:K],
            'output_distances': distances[mask][:K]
        }

In [ ]:
%%time
get_best('Как дела?', label_list, st)

CPU times: user 653 ms, sys: 5.19 ms, total: 658 ms
Wall time: 1.41 s


{'input_text': 'Как дела?',
 'output_texts': array(['how_are_you', 'how_are_you', 'how_are_you'], dtype='<U26'),
 'output_distances': array([0.01165006, 0.01165006, 0.01853184])}

##Модель bert-multilingual-passage-reranking-msmarco

In [ ]:
model_msmarce = Model_searh('amberoad/bert-multilingual-passage-reranking-msmarco')

In [ ]:
model_msmarce.fit(train_list)

In [ ]:
%%time
model_msmarce.get_best('Привет', label_list)

CPU times: user 97.2 ms, sys: 0 ns, total: 97.2 ms
Wall time: 107 ms


{'input_text': 'Привет',
 'output_texts': array(['hello', 'hello', 'hello'], dtype='<U26'),
 'output_distances': array([3.39839268e-13, 3.39839268e-13, 6.63207849e-02])}

##Модель MSTSb_stsb-xlm-r-multilingual

In [ ]:
model_aida = Model_searh('AIDA-UPM/MSTSb_stsb-xlm-r-multilingual')

In [ ]:
model_aida.fit(train_list)

In [ ]:
model_aida.get_best('Расскажи о себе и что ты умеешь?', label_list)

{'input_text': 'Расскажи о себе и что ты умеешь?',
 'output_texts': array(['skills', 'skills', 'acquaintance'], dtype='<U26'),
 'output_distances': array([0.23914132, 0.2692477 , 0.27383558])}

##Модель quora-distilbert-multilingual

In [ ]:
model_quora_multilingual = Model_searh('sentence-transformers/quora-distilbert-multilingual')

In [ ]:
%%time
model_quora_multilingual.fit(train_list)

CPU times: user 15.8 s, sys: 41.3 ms, total: 15.8 s
Wall time: 16.1 s


In [ ]:
model_quora_multilingual.get_best('Расскажи о себе и что ты умеешь?', label_list)

{'input_text': 'Расскажи о себе и что ты умеешь?',
 'output_texts': array(['acquaintance', 'acquaintance', 'acquaintance'], dtype='<U26'),
 'output_distances': array([0.01825803, 0.02029677, 0.02142612])}

##Модель cointegrated/rubert-tiny2

In [ ]:
model_tine = Model_searh('cointegrated/rubert-tiny2')

In [ ]:
model_tine.fit(train_list)

In [ ]:
model_tine.get_best('Привет', label_list)

{'input_text': 'Привет',
 'output_texts': array(['hello', 'hello', 'hello'], dtype='<U26'),
 'output_distances': array([5.46229728e-14, 1.10101614e-01, 1.17256343e-01])}

##Модель rubert-base-cased-nli-threeway

In [ ]:
model_nli_threeway = Model_searh('cointegrated/rubert-base-cased-nli-threeway', typee='nli')

In [ ]:
dct = model_nli_threeway.get_best('Привет', train_list)

In [ ]:
max_dct = max(dct, key=lambda x: dct[x])
print(max_dct, dct[max_dct])

приветствую 0.006456625


In [ ]:
df[df.text == max_dct]

,text,label,Target
994,приветствую,hello,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


##Модель cointegrated/rubert-base-cased-nli-twoway

In [ ]:
model_nli_twoway = Model_searh('cointegrated/rubert-base-cased-nli-twoway', typee='nli')

In [ ]:
model_nli_twoway.get_best('Привет', train_list)

{'input_text': 'Привет',
 'output_texts': array(['no_problem', 'no_problem', 'no_problem', ..., 'angry', 'angry',
        'angry'], dtype='<U26'),
 'output_distances': array([1.7927696e-05, 1.7927696e-05, 1.7927696e-05, ..., 2.3454791e-03,
        2.3454791e-03, 2.3454791e-03], dtype=float32)}

#Ставим оценочки (в разработке)

##quora

In [ ]:
from sklearn import preprocessing

mlb = preprocessing.MultiLabelBinarizer()

In [ ]:
df[['label']].value_counts().head(20)

label             
good                  124
bad                    58
beautiful              51
how_are_you            43
likes_agent            35
thank_you              33
can_you_help           32
bye                    32
clever                 30
my_friend              28
right                  24
needs_advice           24
busy                   23
sorry                  22
no                     21
funny                  20
answer_my_question     20
affront                20
hello                  19
ha_ha                  19
dtype: int64

In [ ]:
df['Target'] = list(mlb.fit_transform(df.label.apply(lambda x: [x])))

In [ ]:
df.Target.head(5)

0    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
1    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
2    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
3    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
4    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: Target, dtype: object

In [ ]:
from sklearn.model_selection import train_test_split
import statistics

X = df.text
y = df.label


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
models_quora_list = [model_aida, model_msmarce, model_quora_multilingual, model_tine]

In [ ]:
%%time
for value in np.arange(20, 21):
    for i, model in enumerate(models_quora_list):
        model.fit(X_train.to_list())
        pred = np.array([statistics.mode(model.get_best(i, label_list, K=value)['output_texts']) for i in X_test]).reshape(-1)
        print(f'model {i}: {sum(pred == y_test.to_numpy()) / len(pred)}')
        print(f'TP/TN: {sum(pred == y_test.to_numpy())}, shape: {len(pred)}')

In [ ]:
model_quora_multilingual.fit(X_train.to_list())

In [ ]:
pred = np.array([statistics.mode(model_quora_multilingual.get_best(i, label_list, max_cos_value=0.2)['output_texts']) for i in X_test]).reshape(-1)

In [ ]:
sum(pred == y_test.to_numpy()) / len(pred)

0.02278481012658228

In [ ]:
temp = pd.DataFrame(columns=['equals'])

temp['pred'] = np.array(pred).reshape(-1)
temp['test'] = y_test.to_numpy()
temp['equals'] = temp.pred == temp.test

In [ ]:
temp['equals'].sum()

11

##NLI

###No train nli model

In [ ]:
from transformers import pipeline
p = pipeline(
  task='zero-shot-classification',
  model='cointegrated/rubert-base-cased-nli-twoway'
)

In [ ]:
noo_train = p(
  sequences="Привет, как дела?",
  candidate_labels=','.join(train_list),
  hypothesis_template="{}."
)

In [ ]:
np.array(label_list)[np.argsort(np.array(noo_train['scores']))]

array(['talk_to_me', 'talk_to_me', 'talk_to_me', ..., 'good', 'good',
       'good'], dtype='<U26')

##With train nli model

In [ ]:
test = ['Привет, как дела?', 'Расскажи анекдот', 'Что ты умеешь? Анекдот', 'Спасибо за ответ!', 'Спасибо за ответ! Как дела?']

In [ ]:
for text in test:
    print(model_nli_threeway.get_best(text, train_list, max_value=0.2))

{'input_text': 'Привет, как дела?', 'output_texts': array(['скажи', 'пожалуйста', 'отвечай на вопрос', ..., 'мне всё равно',
       'ты абсолютно бесполезен', 'вообще всё равно'], dtype='<U47'), 'output_distances': array([9.2038931e-03, 8.4990300e-03, 8.4824236e-03, ..., 8.4861758e-06,
       6.6617513e-06, 6.3733041e-06], dtype=float32)}
{'input_text': 'Расскажи анекдот', 'output_texts': array(['расскажи анекдот', 'скажи', 'поговори', ..., 'вообще-то нет',
       'ты мне совсем не помогаешь', 'ты абсолютно бесполезен'],
      dtype='<U47'), 'output_distances': array([1.0960256e-02, 1.0184640e-02, 1.0076572e-02, ..., 4.5513693e-06,
       3.6159095e-06, 3.4986758e-06], dtype=float32)}
{'input_text': 'Что ты умеешь? Анекдот', 'output_texts': array(['что ты умеешь?', 'Что ты умеешь?', 'чем ты занимаешься', ...,
       'я очень сонный', 'ты просто ужас', 'ты абсолютно бесполезен'],
      dtype='<U47'), 'output_distances': array([7.5233830e-03, 7.5233830e-03, 7.2600706e-03, ..., 4.1539161e